In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rectal_Cancer"
cohort = "GSE123390"

# Input paths
in_trait_dir = "../../input/GEO/Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Rectal_Cancer/GSE123390"

# Output paths
out_data_file = "../../output/preprocess/Rectal_Cancer/GSE123390.csv"
out_gene_data_file = "../../output/preprocess/Rectal_Cancer/gene_data/GSE123390.csv"
out_clinical_data_file = "../../output/preprocess/Rectal_Cancer/clinical_data/GSE123390.csv"
json_path = "../../output/preprocess/Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Locally advanced rectal cancer transcriptomic-based secretome analysis according to neoadjuvant chemoradiotherapy response"
!Series_summary	"Most patients with locally advanced rectal cancer (LARC) present incomplete pathological response (pIR) to neoadjuvant chemoradiotherapy (nCRT). Despite the efforts to predict treatment response using tumor-molecular features, as differentially expressed genes, no molecule has proved to be a strong biomarker. The tumor secretome analysis is a promising strategy for biomarkers identification, which can be assessed using transcriptomic data. Here, we performed transcriptomic-based secretome analysis to select potentially secreted proteins using an in silico approach. The tumor expression profile of 28 LARC biopsies carefully selected and collected before nCRT was compared with normal rectal tissues (NT). The expression profile showed no significant differences between cases with complete (pCR) and incomplete re

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
#1. Check gene expression data availability
is_gene_available = True  # This is Affymetrix expression data (Human Transcriptome Array 2.0)

#2. Variable Availability and Data Type Conversion
#2.1 Identify rows containing trait, age, and gender data
trait_row = 2  # The trait is response to treatment (pCR/pIR) in row 2
age_row = None  # Age is not available in the sample characteristics
gender_row = None  # Gender is not available in the sample characteristics

#2.2 Data Type Conversion functions
def convert_trait(value):
    """Convert treatment response to binary values: 
    pCR (pathological Complete Response) = 1, 
    pIR (pathological Incomplete Response) = 0,
    Other/unknown = None"""
    if not value or not isinstance(value, str):
        return None
    
    # Extract value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert response values
    if value == 'pCR':
        return 1
    elif value == 'pIR':
        return 0
    else:
        return None

def convert_age(value):
    # Age data is not available, but function defined for completeness
    return None

def convert_gender(value):
    # Gender data is not available, but function defined for completeness
    return None

#3. Save Metadata - Initial Filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

#4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'GSM3502511': [0.0], 'GSM3502512': [0.0], 'GSM3502513': [0.0], 'GSM3502514': [0.0], 'GSM3502515': [0.0], 'GSM3502516': [0.0], 'GSM3502517': [1.0], 'GSM3502518': [1.0], 'GSM3502519': [0.0], 'GSM3502520': [1.0], 'GSM3502521': [0.0], 'GSM3502522': [0.0], 'GSM3502523': [0.0], 'GSM3502524': [0.0], 'GSM3502525': [0.0], 'GSM3502526': [0.0], 'GSM3502527': [0.0], 'GSM3502528': [1.0], 'GSM3502529': [1.0], 'GSM3502530': [0.0], 'GSM3502531': [0.0], 'GSM3502532': [0.0], 'GSM3502533': [1.0], 'GSM3502534': [1.0], 'GSM3502535': [1.0], 'GSM3502536': [1.0], 'GSM3502537': [1.0], 'GSM3502538': [1.0], 'GSM3502539': [nan], 'GSM3502540': [nan], 'GSM3502541': [nan], 'GSM3502542': [nan], 'GSM3502543': [nan]}
Clinical data saved to ../../output/preprocess/Rectal_Cancer/clinical_data/GSE123390.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These gene identifiers (like "2824546_st") appear to be probe IDs from an Affymetrix microarray
# They are not standard human gene symbols and would need to be mapped to gene symbols

# These look like Affymetrix probe IDs which need to be mapped to official gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# Looking at the gene expression identifiers and comparing with the annotation data,
# we need to map from probe IDs in the gene expression data to gene symbols

# After examining both dataframes, it seems "ID" in the annotation dataframe is our key 
# for gene identifiers and "gene_assignment" contains gene symbols

# 1. Extract the mapping from the gene annotation dataframe
gene_mapping = get_gene_mapping(
    annotation=gene_annotation,
    prob_col='ID',
    gene_col='gene_assignment'
)

# 2. Apply the gene mapping to convert probe-level measurements to gene expression
# This will handle the many-to-many mapping by properly distributing signal
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)

# Preview the mapped gene data
print("Gene expression data after mapping to gene symbols:")
print(f"Number of genes: {len(gene_data)}")
print(f"Number of samples: {gene_data.shape[1]}")
print("First few gene symbols:")
print(gene_data.index[:10])


Gene expression data after mapping to gene symbols:
Number of genes: 71528
Number of samples: 33
First few gene symbols:
Index(['A-', 'A-2', 'A-52', 'A-575C2', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V',
       'A0'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Get clinical data from previous step
# Use the clinical data that was already extracted and saved in Step 2
clinical_file_path = out_clinical_data_file
if os.path.exists(clinical_file_path):
    clinical_features = pd.read_csv(clinical_file_path, index_col=0)
    print(f"Clinical features loaded from {clinical_file_path}")
    print(f"Clinical features shape: {clinical_features.shape}")
else:
    # If file doesn't exist, we need to extract it again
    # Get trait data as described in step 2, where trait_row=2 and age/gender are not available
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=2,  # 'response: pIR/pCR' is in row 2
        convert_trait=convert_trait,  # Use the previously defined function
        age_row=None,  # No age data available
        convert_age=None,
        gender_row=None,  # No gender data available
        convert_gender=None
    )
    print(f"Clinical features re-extracted")
    print(f"Clinical features shape: {clinical_features.shape}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(f"First few columns: {list(linked_data.columns[:5])}")

# 3. Handle missing values in the linked data
linked_data_processed = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data_processed.shape}")

# Add validation check - if no samples remain, note the issue
if linked_data_processed.shape[0] == 0:
    print("No samples remain after handling missing values. The dataset cannot be processed further.")
    is_trait_biased = True  # Mark as biased since we can't use it
    unbiased_linked_data = linked_data_processed
else:
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_processed, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from rectal cancer patients with treatment response data (pCR/pIR)."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (24018, 33)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1', 'A2ML1-AS2', 'A2MP1', 'A4GALT']


Normalized gene data saved to ../../output/preprocess/Rectal_Cancer/gene_data/GSE123390.csv
Clinical features loaded from ../../output/preprocess/Rectal_Cancer/clinical_data/GSE123390.csv
Clinical features shape: (1, 33)
Linked data shape: (33, 24019)
First few columns: ['Rectal_Cancer', 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M']


Shape after handling missing values: (28, 24019)
For the feature 'Rectal_Cancer', the least common label is '1.0' with 11 occurrences. This represents 39.29% of the dataset.
The distribution of the feature 'Rectal_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Rectal_Cancer/GSE123390.csv
